<a href="https://colab.research.google.com/github/pserebrennikov/3rd-year-project/blob/master/2_second_order_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 2 - Second order methods
### Course on Optimization for Machine Learning - Dr. F. Ballarin
### Master Degree in Data Analytics for Business, Catholic University of the Sacred Heart, Milano

In this notebook we implement some second order methods for unconstrained optimization.

In [ ]:
import typing

In [ ]:
import numpy as np
import plotly.colors
import plotly.graph_objects as go

## Exercise 2.1 (continuation of Exercises 1.1 and 1.5)
Let $\boldsymbol{w} \in \mathbb{R}^2$. Consider the *Booth function*
$$f(\boldsymbol{w}) = (w^{(0)} + 2 w^{(1)} - 7)^2 + (2 w^{(0)} + w^{(1)} - 5)^2.$$

10. Introduce a new variable $\widetilde{\boldsymbol{w}}$
$$ \widetilde{\boldsymbol{w}} = \boldsymbol{A} \boldsymbol{w} $$
where
$$ \boldsymbol{A} = \sqrt{2} \begin{bmatrix}2 & 1\\1 & 2\end{bmatrix}.$$
Denote by $\widetilde{f}$ the Booth function with respect to the new variable
$$\widetilde{f}(\widetilde{\boldsymbol{w}}) = f(\boldsymbol{A}^{-1} \widetilde{\boldsymbol{w}})$$
Draw a contour plot of the function $\widetilde{f}$ on the square domain $[-3, 17] \times [0, 20]$ (in $\widetilde{\boldsymbol{w}}$).

*Solution*:
> We define a Python function for the evaluation $\widetilde{f}$. Since the input variable is now $\widetilde{f}$, we first transform it to $\boldsymbol{w}$, and use the previous implementation.

In [ ]:
A = np.sqrt(2) * np.array([[2, 1], [1, 2]])
A_inv = np.linalg.inv(A)

In [ ]:
def f_tilde_ex_2_1(w_tilde: np.ndarray) -> float:
    r"""Evaluate \tilde{f}(w)."""
    w = np.dot(A_inv, w_tilde)  # Watch out! np.dot vs * with matrix/vector operations
    return (w[0] + 2 * w[1] - 7)**2 + (2 * w[0] + w[1] - 5)**2

> Afterwards we prepare the contour plot.

In [ ]:
domain_tilde_component_0 = [-3, 17]
domain_tilde_component_1 = [0, 20]

In [ ]:
w_tilde_component_0 = np.linspace(domain_tilde_component_0[0], domain_tilde_component_0[1], 100)
w_tilde_component_1 = np.linspace(domain_tilde_component_1[0], domain_tilde_component_1[1], 100)

In [ ]:
f_tilde_w_tilde = np.zeros((len(w_tilde_component_0), len(w_tilde_component_1)))
for i in range(f_tilde_w_tilde.shape[0]):
    for j in range(f_tilde_w_tilde.shape[1]):
        f_tilde_w_tilde[j, i] = f_tilde_ex_2_1([w_tilde_component_0[i], w_tilde_component_1[j]])

In [ ]:
fig = go.Figure(data=[go.Contour(x=w_tilde_component_0, y=w_tilde_component_1, z=f_tilde_w_tilde)])
fig.update_layout(
    title="Function exercise 2.1 - contour plot in the w_tilde variables",
    width=512, height=512, autosize=False)
fig.show()

> Notice how the contour lines of $\widetilde{f}$ are circles, while the ones of $f$ were very elongated ellipses.

11. Recall that the gradient of $\nabla f = \nabla_{\boldsymbol{w}}f $  is
\begin{equation*}
\nabla f(\boldsymbol{w}) = \begin{bmatrix}
10 w^{(0)} + 8 w^{(1)} - 34\\
8 w^{(0)} + 10 w^{(1)} - 38\\
\end{bmatrix},
\end{equation*}
the hessian $\nabla^2 f = \nabla_\boldsymbol{w}^2 f$ is
\begin{equation*}
\nabla^2 f(\boldsymbol{w}) =
\begin{bmatrix}
10 & 8\\
8 & 10\\
\end{bmatrix}
\end{equation*}
and the global minima of $f$ is $\boldsymbol{w}^* = (1,3)$. Determine the corresponding expressions for $\nabla \widetilde{f} = \nabla_{\widetilde{\boldsymbol{w}}} \widetilde{f}$, $\nabla^2 \widetilde{f} = \nabla_{\widetilde{\boldsymbol{w}}}^2 \widetilde{f}$ and the global minima $\widetilde{\boldsymbol{w}}^*$ of $\widetilde{f}$.

*Solution*:
> From the chain rule in calculus we have that
$$ \nabla_{\widetilde{\boldsymbol{w}}} \widetilde{f}|_{\widetilde{\boldsymbol{w}}} = 
\boldsymbol{A}^{-1} \ \nabla_{\boldsymbol{w}}f|_{\boldsymbol{w} = \boldsymbol{A}^{-1} \widetilde{\boldsymbol{w}}}$$
Therefore, when implementing in Python a function for the evaluation of $\nabla_{\widetilde{\boldsymbol{w}}} \widetilde{f}$, we should
> 1. transform the input $\widetilde{\boldsymbol{w}}$ into $\boldsymbol{w} = \boldsymbol{A}^{-1} \widetilde{\boldsymbol{w}}$,
> 2. pass the value of $\boldsymbol{w}$ to the evaluation of $\nabla_{\boldsymbol{w}}f$,
> 3. premultiply the result of such evaluation by $\boldsymbol{A}^{-1}$.

In [ ]:
def grad_f_tilde_ex_2_1(w_tilde: np.ndarray) -> np.ndarray:
    r"""Evaluate \nabla \tilde{f}(w)."""
    w = np.dot(A_inv, w_tilde)
    grad_f = np.array([10 * w[0] + 8 * w[1] - 34, 8 * w[0] + 10 * w[1] - 38])
    return np.dot(A_inv, grad_f)

> The global minima $\widetilde{\boldsymbol{w}}^*$ of $\widetilde{f}$ can be obtained from $\widetilde{\boldsymbol{w}}^*$ by using the transformation of variable formula
$$ \widetilde{\boldsymbol{w}} = \boldsymbol{A} \boldsymbol{w} $$

In [ ]:
w_tilde_star = np.dot(A, np.array([1, 3]))
w_tilde_star

> We check our implementation of $\widetilde{f}$ and $\nabla_{\widetilde{\boldsymbol{w}}} \widetilde{f}$ by evaluating them at $\widetilde{\boldsymbol{w}}^*$. Both should give zero as an answer.

In [ ]:
f_tilde_ex_2_1(w_tilde_star)

In [ ]:
grad_f_tilde_ex_2_1(w_tilde_star)

> Taking a further derivative and applying the chain rule again results in
$$ \nabla_{\widetilde{\boldsymbol{w}}}^2 \widetilde{f}|_{\widetilde{\boldsymbol{w}}} = 
\boldsymbol{A}^{-2} \ \nabla_{\boldsymbol{w}}^2f|_{\boldsymbol{w} = \boldsymbol{A}^{-1} \widetilde{\boldsymbol{w}}}.$$
Since $\nabla_{\boldsymbol{w}}^2f$ is constant we can simply compute

In [ ]:
A_minus_2 = np.dot(A_inv, A_inv)
hessian_f_tilde = np.dot(A_minus_2, np.array([[10, 8], [8, 10]]))
hessian_f_tilde

12. Apply the gradient descent with the value of $\alpha$ suggested by the convergence theory and any choice of $\varepsilon$ and $\widetilde{\boldsymbol{w}}_0$. How many iterations does it take to converge to $\widetilde{\boldsymbol{w}}^*$?

*Solution*:
> We copy the implementation of the gradient descent method from the first tutorial.

In [ ]:
def gradient_descent_ex_2_1(alpha: float, epsilon: float, w_tilde_0: np.ndarray) -> typing.Tuple[
        np.ndarray, np.ndarray, np.ndarray]:
    """
    Run the gradient descent method with constant step length.

    Parameters
    ----------
    alpha : float
        constant step length.
    epsilon : float
        tolerance for the stopping criterion on the error on the cost.
    w_tilde_0 : 1d numpy array
        numpy array containing the initial condition.

    Returns
    -------
    2d numpy array
        history of the optimization variables iterations.
    1d numpy array
        history of the cost function values.
    2d numpy array
        history of the gradient of the cost function.
    """
    # Prepare lists collecting the required outputs over the iterations
    all_w_tilde = [w_tilde_0]
    all_f_tilde = [f_tilde_ex_2_1(w_tilde_0)]
    all_grad_f_tilde = [grad_f_tilde_ex_2_1(w_tilde_0)]

    # Prepare iteration counter
    k = 0

    # Use the error on the cost to determine when the while loop should stop.
    while all_f_tilde[k] > epsilon:
        w_tilde_k = all_w_tilde[k]
        grad_f_tilde_k = all_grad_f_tilde[k]
        w_tilde_k_plus_1 = w_tilde_k - alpha * grad_f_tilde_k

        # Update required outputs
        all_w_tilde.append(w_tilde_k_plus_1)
        all_f_tilde.append(f_tilde_ex_2_1(w_tilde_k_plus_1))
        all_grad_f_tilde.append(grad_f_tilde_ex_2_1(w_tilde_k_plus_1))

        # Bail out if the descent condition is not satisfied
        if all_f_tilde[k + 1] >= all_f_tilde[k]:
            print("WARNING: descent conditions is not satisfied")
            break

        # Increment iteration counter
        k += 1

    # For convenience we transform the outputs into numpy array before returning
    return np.array(all_w_tilde), np.array(all_f_tilde), np.array(all_grad_f_tilde)

> From the expression of $\nabla_{\widetilde{\boldsymbol{w}}}^2 \widetilde{f}$ we get that the smoothness parameter is $L = 1$, because the matrix is diagonal (therefore its eigenvalues are on the diagonal) and the maximum eigenvalue (actually, both of them) is 1. So we choose $\alpha = 1$.

In [ ]:
all_w_tilde, all_f_tilde, all_grad_f_tilde = gradient_descent_ex_2_1(1, 1e-5, np.array([1.0, 2.0]))

In [ ]:
all_w_tilde

> The gradient method applied to $\widetilde{f}$ converges in one iteration. Try to change $\varepsilon$ and $\widetilde{\boldsymbol{w}}_0$ and confirm that just one iteration will be required regardless of your choice.
> By changing the optimization variable we have managed to improve over all the methods discussed in the lecture 1, and get convergence in just one iteration!
>
> Why does it take only 1 iteration? Recall from the convergence theory that the gradient method is lineary convergent when applied to $L$-smooth and $\mu$-strongly convex functions, and with $\alpha = 1/L$ the following formula holds
$$ f(\boldsymbol{w}_{k+1}) - f(\boldsymbol{w}^*) \leq \left(1 - \frac{\mu}{L}\right) \left(f(\boldsymbol{w}_k) - f(\boldsymbol{w}^*)\right). $$
> Here we have $\mu = L = 1$, so
$$ 1 - \frac{\mu}{L} = 0.$$
Therefore, regardless of the initial error
$$ f(\boldsymbol{w}_0) - f(\boldsymbol{w}^*),$$
the function error at the first iteration will be zero.

## Exercise 2.2
Let $\boldsymbol{w} \in \mathbb{R}^2$. Consider the following functions:
* the *Rosenbrock function* defined as
$$r(\boldsymbol{w}) = 10 \left(w^{(1)} - [w^{(0)}]^2\right)^2 + (1 - w^{(0)})^2,$$
* the *three-hump Camel function* defined as
$$h(\boldsymbol{w}) = 2 [w^{(0)}]^2 - 1.05 [w^{(0)}]^4 + \frac{1}{6} [w^{(0)}]^6 + w^{(0)} w^{(1)} + [w^{(1)}]^2.$$

1. Draw a contour plot of the functions $r$ and $h$ on the square domain $[-2, 2]^2$.

*Solution*:

In [ ]:
domain_component_0 = [-2, 2]
domain_component_1 = [-2, 2]

In [ ]:
w_component_0 = np.linspace(domain_component_0[0], domain_component_0[1], 200)
w_component_1 = np.linspace(domain_component_1[0], domain_component_1[1], 200)

In [ ]:
def r_ex_2_2(w: np.ndarray) -> float:
    """Evaluate r(w)."""
    return 10 * (w[1] - w[0]**2)**2 + (1 - w[0])**2

In [ ]:
def h_ex_2_2(w: np.ndarray) -> float:
    """Evaluate h(w)."""
    return 2 * w[0]**2 - 1.05 * w[0]**4 + 1 / 6 * w[0]**6 + w[0] * w[1] + w[1]**2

In [ ]:
r_w = np.zeros((len(w_component_0), len(w_component_1)))
h_w = np.zeros((len(w_component_0), len(w_component_1)))
for i in range(r_w.shape[0]):
    for j in range(r_w.shape[1]):
        r_w[j, i] = r_ex_2_2([w_component_0[i], w_component_1[j]])
        h_w[j, i] = h_ex_2_2([w_component_0[i], w_component_1[j]])

In [ ]:
fig = go.Figure(data=[go.Contour(
    x=w_component_0, y=w_component_1, z=np.log10(r_w),
    hovertemplate="x: %{x:.2f}<br>y: %{y:.2f}<br>z: 10^%{z:.2f} = %{customdata}", customdata=r_w,
    colorbar=dict(tickprefix="10^")
)])
fig.update_layout(
    title="Rosenbrock function - contour plot (log scale)", width=512, height=512, autosize=False)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Contour(
    x=w_component_0, y=w_component_1, z=np.log10(h_w),
    hovertemplate="x: %{x:.2f}<br>y: %{y:.2f}<br>z: 10^%{z:.2f} = %{customdata}", customdata=h_w,
    colorbar=dict(tickprefix="10^")
)])
fig.update_layout(
    title="Three-hump Camel function - contour plot (log scale)", width=512, height=512, autosize=False)
fig.show()

2. Compute the gradient $\nabla r$, the hessian $\nabla^2 r$ and determine the global minimum of the function $r$.

*Solution*:
> We compute the partial derivatives of $r(\boldsymbol{w}) = 10 \left(w^{(1)} - [w^{(0)}]^2\right)^2 + (1 - w^{(0)})^2$ to obtain the gradient
\begin{equation*}
\nabla r(\boldsymbol{w}) = \begin{bmatrix}
-40 w^{(0)} \left(w^{(1)} - [w^{(0)}]^2\right) - 2 (1 - w^{(0)})\\
20 \left(w^{(1)} - [w^{(0)}]^2\right)
\end{bmatrix}
\end{equation*}
and the hessian
\begin{equation*}
\nabla^2 r(\boldsymbol{w}) = \begin{bmatrix}
120 [w^{(0)}]^2 - 40 w^{(1)} +2 & - 40 w^{(0)}\\
-40 w^{(0)} & 20
\end{bmatrix}.
\end{equation*}
By solving $\nabla r(\boldsymbol{w}) = 0$ we see that $\boldsymbol{w}_r^* = (1, 1)$ is the only stationary point.

In [ ]:
w_star_r = np.array([1, 1])
w_star_r

In [ ]:
r_ex_2_2(w_star_r)

In [ ]:
def grad_r_ex_2_2(w: np.ndarray) -> np.ndarray:
    r"""Evaluate \nabla r(w)."""
    return np.array([
        - 40 * w[0] * (w[1] - w[0]**2) - 2 * (1 - w[0]),
        20 * (w[1] - w[0]**2)
    ])

In [ ]:
grad_r_ex_2_2(w_star_r)

> We check the sufficient condition for $\boldsymbol{w}^*$ to be (the global) minimum.

In [ ]:
def hessian_r_ex_2_2(w: np.ndarray) -> np.ndarray:
    r"""Evaluate \nabla^2 r(w)."""
    return np.array([
        [120 * w[0]**2 - 40 * w[1] + 2, - 40 * w[0]],
        [- 40 * w[0], 20]
    ])

In [ ]:
eigs, _ = np.linalg.eig(hessian_r_ex_2_2(w_star_r))
eigs

In [ ]:
assert (eigs > 0).all()

> Notice how the eigenvalues at the global minimum are almost three orders of magnitude apart. This indicates that the minimum is in a very narrow valley.

3. Compute the gradient $\nabla h$, the hessian $\nabla^2 h$ and determine the global minimum of the function $h$.

*Solution*:
> We compute the partial derivatives of $h(\boldsymbol{w}) = 2 [w^{(0)}]^2 - 1.05 [w^{(0)}]^4 + \frac{1}{6} [w^{(0)}]^6 + w^{(0)} w^{(1)} + [w^{(1)}]^2$ to obtain the gradient
\begin{equation*}
\nabla h(\boldsymbol{w}) = \begin{bmatrix}
4 w^{(0)} - 4.2 [w^{(0)}]^3 + [w^{(0)}]^5 + w^{(1)}\\
w^{(0)} + 2 w^{(1)}
\end{bmatrix}
\end{equation*}
and the hessian
\begin{equation*}
\nabla^2 h(\boldsymbol{w}) = \begin{bmatrix}
4 - 12.6 [w^{(0)}]^2 + 5 [w^{(0)}]^4 & 1\\
1 & 2
\end{bmatrix}.
\end{equation*}
The system $\nabla r(\boldsymbol{w}) = 0$ results in
\begin{equation*}
\begin{cases}
3.5 w^{(0)}  - 4.2 [w^{(0)}]^3 + [w^{(0)}]^5 = 0\\
w^{(1)} = - 0.5 w^{(0)}
\end{cases}
\end{equation*}
We can easily solve the polynomial equation in the first line by collecting the common factor $w^{(0)}$ and reducing the resulting fourth order term to a second order equation. Alternatively, we query the rootfinding facilities for polynomials provided in [`numpy.roots`](https://numpy.org/doc/stable/reference/generated/numpy.roots.html).

In [ ]:
roots = np.roots([1, 0, -4.2, 0, 3.5, 0])
roots = np.sort(roots)
roots

> The function $h$ has 5 stationary points, which we collect in a `numpy array` below.

In [ ]:
w_star_h = np.vstack([roots, - 0.5 * roots]).T
w_star_h

> We determine the corresponding function $h$ values at each stationary point by means of a `for` loop.

In [ ]:
for row_h in range(w_star_h.shape[0]):
    print("h(" + str(w_star_h[row_h, :]) + " = " + str(h_ex_2_2(w_star_h[row_h, :])))

> We implement a Python function for the evaluation of $\nabla h$, and test it on the stationary points.

In [ ]:
def grad_h_ex_2_2(w: np.ndarray) -> np.ndarray:
    r"""Evaluate \nabla h(w)."""
    return np.array([
        4 * w[0] - 4.2 * w[0]**3 + w[0]**5 + w[1],
        w[0] + 2 * w[1]
    ])

In [ ]:
for row_h in range(w_star_h.shape[0]):
    print("grad h(" + str(w_star_h[row_h, :]) + " = " + str(grad_h_ex_2_2(w_star_h[row_h, :])))

> Finally, we implement a Python function for the evaluation of $\nabla^2 h$, and use it to determine the nature (minimum, maximum, saddle) of the stationary points.

In [ ]:
def hessian_h_ex_2_2(w: np.ndarray) -> np.ndarray:
    r"""Evaluate \nabla^2 h(w)."""
    return np.array([
        [4 - 12.6 * w[0]**2 + 5 * w[0]**4, 1],
        [1, 2]
    ])

In [ ]:
for row_h in range(w_star_h.shape[0]):
    eigs, _ = np.linalg.eig(hessian_h_ex_2_2(w_star_h[row_h, :]))
    print("eigenvalues of hessian h(" + str(w_star_h[row_h, :]) + " = " + str(eigs))

> From these information we determine that there is a global minimum at $(0, 0)$, two addiational local minima, and two saddle points.

4. Implement minimization of a function $f$ using the gradient descent or the Newton method with backtracking line search in a Python function. Such function should:
   * take as input the method to use (`Newton` or `gradient`), the function $f$, its gradient $\nabla f$, its hessian $\nabla^2 f$, the constants $\alpha$, $c_1$ and $c_2$ of the backtracking algorithm, the tolerance $\varepsilon$ for the stopping criterion and the initial condition $\boldsymbol{w}_{0}$;
   * return as outputs the optimization variable iterations $\{\boldsymbol{w}_k\}_k$, the corresponding function values $\{f(\boldsymbol{w}_k)\}_k$ and gradients $\{\nabla f(\boldsymbol{w}_k)\}_k$.

   Use the following stopping criteria:
   * the main stopping criterion is based on the norm of the gradient. Continue the iterations while such norm is above $\varepsilon$;
   * however, when applying the Newton method in combination with backtracking, it may well be possible that search direction $\boldsymbol{p}_k$ is *not* a descent direction for some iterations $k$, and backtracking will finish only when $\alpha_k = 0$. To avoid getting stuck in an infinite loop, we adapt the following safeguard: if the norm of the increment between two iterations is below the tolerance $\varepsilon$ then terminate early, and print a warning. Similar safeguards to avoid infinite loops are very common in practical implementations of the methods we see in this course. 

*Solution*:
> We start from the implementation of the gradient descent with bracktracking we discussed in Tutorial 1, and change the definition of the search direction $\boldsymbol{p}_k$ according to the `method` input. 

In [ ]:
def optimization_with_backtracking_line_search_ex_2_2(
    method: str, f: typing.Callable, grad_f: typing.Callable, hess_f: typing.Callable, alpha: float,
    c_1: float, c_2: float, epsilon: float, w_0: np.ndarray
) -> typing.Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Run the gradient descent method or Newton method with backtracking line search.

    Parameters
    ----------
    method : str
        optimization method to be used, either "gradient" or "Newton".
    f, grad_f, hess_f : Python function
        callable evaluating the cost function, its gradient and its hessian, respectively.
    alpha : float
        initial step length.
    c_1, c_2 : float
        constants of the backtracking algorithm.
    epsilon : float
        tolerance for the stopping criterion on the error on the norm of the gradient of the cost.
    w_0 : 1d numpy array
        numpy array containing the initial condition.

    Returns
    -------
    2d numpy array
        history of the optimization variables iterations.
    1d numpy array
        history of the cost function values.
    2d numpy array
        history of the gradient of the cost function.
    """
    assert method in ("gradient", "Newton")

    # Prepare lists collecting the required outputs over the iterations
    all_w = [w_0]
    all_f = [f(w_0)]
    all_grad_f = [grad_f(w_0)]

    # Prepare iteration counter
    k = 0

    # Use the norm of the gradient as stopping criterion.
    while np.linalg.norm(all_grad_f[k]) > epsilon:
        w_k = all_w[k]
        f_k = all_f[k]
        grad_f_k = all_grad_f[k]

        # Define the search direction p_k
        if method == "gradient":
            p_k = - grad_f_k
            p_k_dot_g_k = - np.linalg.norm(grad_f_k)**2
        elif method == "Newton":
            hess_f_k = hess_f(w_k)
            p_k = - np.linalg.solve(hess_f_k, grad_f_k)
            p_k_dot_g_k = np.dot(p_k, grad_f_k)

        # Carry out a backtracking line search
        alpha_k = alpha
        while f(w_k + alpha_k * p_k) > f_k + c_1 * alpha_k * p_k_dot_g_k:
            alpha_k = c_2 * alpha_k

        # Compute w_{k+1}
        w_k_plus_1 = w_k + alpha_k * p_k

        # Update required outputs
        all_w.append(w_k_plus_1)
        all_f.append(f(w_k_plus_1))
        all_grad_f.append(grad_f(w_k_plus_1))

        # Increment iteration counter
        k += 1

        # Bail out if backtracking failed
        if np.linalg.norm(all_grad_f[k] - all_grad_f[k - 1]) < epsilon:
            print("WARNING: " + method + " method terminated early due to not having found a descent step. "
                  + "Backtracking terminated with alpha_k = " + str(alpha_k))
            break

    # For convenience we transform the outputs into numpy array before returning
    return np.array(all_w), np.array(all_f), np.array(all_grad_f)

5. Choose $\alpha = 1$, $c_1 = 0.1$, $c_2 = 0.7$, $\varepsilon = 10^{-5}$, $\boldsymbol{w}_0 = (-1, 1.5)$, and the Rosenbrock function $r$. Run gradient and Newton methods, and visualize a semilogarithimic plot of error in the function value $\{r(\boldsymbol{w}_k) - r(\boldsymbol{w}^*)\}_k$ versus the iteration counter $k$.

*Solution*:
> We run our previous implementation.

In [ ]:
all_w_gradient, all_r_gradient, all_grad_r_gradient = optimization_with_backtracking_line_search_ex_2_2(
    "gradient", r_ex_2_2, grad_r_ex_2_2, hessian_r_ex_2_2, 1, 0.1, 0.7, 1e-5, np.array([-1.0, 1.5]))

In [ ]:
all_w_gradient.shape[0]

In [ ]:
all_w_gradient[-1, :]

In [ ]:
all_w_newton, all_r_newton, all_grad_r_newton = optimization_with_backtracking_line_search_ex_2_2(
    "Newton", r_ex_2_2, grad_r_ex_2_2, hessian_r_ex_2_2, 1, 0.1, 0.7, 1e-5, np.array([-1.0, 1.5]))

In [ ]:
all_w_newton.shape[0]

In [ ]:
all_w_newton[-1, :]

> Both methods converge to the global minimum. However, Newton method converges in roughly 10 iterations, while the gradient method takes more than 1000 iterations.

> The semilog plot of the error on the function value clearly shows this fast convergence. Compared to the line of the gradient method, the line corresponding to the Newton method is almost vertical close to convergence. To better appreciate this uncomment the line that restricts the horizontal axis to the interval $[0, 15]$.

In [ ]:
fig = go.Figure()
all_methods = ["Gradient", "Newton"]
all_r_method = [all_r_gradient, all_r_newton]
for method_index in range(2):
    fig.add_scatter(
        x=np.arange(all_r_method[method_index].shape[0]), y=all_r_method[method_index],
        marker=dict(color=plotly.colors.qualitative.Set1[method_index], size=10),
        line=dict(color=plotly.colors.qualitative.Set1[method_index], width=2),
        mode="lines+markers", name=all_methods[method_index] + " method"
    )
fig.update_layout(
    title="Rosenbrock function - error on the function value - different methods",
    width=768, height=768, autosize=False
)
# fig.update_xaxes(range=[0, 15])
fig.update_yaxes(type="log", exponentformat="power")
fig.show()

> With this case we have concluded the graphical representation of the concepts of sublinear, linear and superlinear convergence. The curve of the error measure in a semilog plot, at least for $k$ big enough:
> * resembles an almost horizontal line when there is sublinear convergence (that we had seen in Exercise 1.3, as motivation for acceleration techniques);
> * resembles an almost vertical line when there is superlinear convergence (as the quadratic convergence of the Newton method in this exercise);
> * resembles a "diagonal" line when there is linear convergence corresponds (as the convergence of the gradient method in this exercise).

6. Choose $\alpha = 1$, $c_1 = 0.1$, $c_2 = 0.7$, $\varepsilon = 10^{-5}$, two choices for $\boldsymbol{w}_0 = $ $(-1, 0.7)$ or $(0.7, 0.7)$, and the three-hump Camel function $h$. To which stationary point do the gradient and Newton methods converge?

*Solution*:
> We run our previous implementation $\boldsymbol{w}_0 = (-1, 0.7)$.

In [ ]:
all_w_gradient, all_h_gradient, all_grad_h_gradient = optimization_with_backtracking_line_search_ex_2_2(
    "gradient", h_ex_2_2, grad_h_ex_2_2, hessian_h_ex_2_2, 1, 0.1, 0.7, 1e-5, np.array([-1.0, 0.7]))

In [ ]:
all_w_gradient.shape[0]

In [ ]:
all_w_gradient[-1, :]

In [ ]:
all_w_newton, all_h_newton, all_grad_h_newton = optimization_with_backtracking_line_search_ex_2_2(
    "Newton", h_ex_2_2, grad_h_ex_2_2, hessian_h_ex_2_2, 1, 0.1, 0.7, 1e-5, np.array([-1.0, 0.7]))

In [ ]:
all_w_newton.shape[0]

In [ ]:
all_w_newton[-1, :]

> From the same initial point $\boldsymbol{w}_0 = (-1, 0.7)$, the gradient method converged to the global minimum, while the Newton method got stuck very close to a saddle point, unable to find a descent direction anymore.

> We next run our previous implementation $\boldsymbol{w}_0 = (0.7, 0.7)$.

In [ ]:
all_w_gradient, all_h_gradient, all_grad_h_gradient = optimization_with_backtracking_line_search_ex_2_2(
    "gradient", h_ex_2_2, grad_h_ex_2_2, hessian_h_ex_2_2, 1, 0.1, 0.7, 1e-5, np.array([0.7, 0.7]))

In [ ]:
all_w_gradient.shape[0]

In [ ]:
all_w_gradient[-1, :]

In [ ]:
all_w_newton, all_h_newton, all_grad_h_newton = optimization_with_backtracking_line_search_ex_2_2(
    "Newton", h_ex_2_2, grad_h_ex_2_2, hessian_h_ex_2_2, 1, 0.1, 0.7, 1e-5, np.array([0.7, 0.7]))

In [ ]:
all_w_newton.shape[0]

In [ ]:
all_w_newton[-1, :]

> From the same initial point $\boldsymbol{w}_0 = (0.7, 0.7)$, the gradient method converged to the global minimum, while Newton method converged to a local minimum.

7. Apply both gradient and Newton method starting from $6^2$ different inital conditions, equispaced in the domain $[-2, 2]$. Compare the two methods in terms of the number of runs that converge to the global minimum, local minima and saddle point.

In [ ]:
solutions = {"gradient": dict(), "Newton": dict()}
for i in np.linspace(-2, 2, 6):
    for j in np.linspace(-2, 2, 6):
        all_w_gradient, _, _ = optimization_with_backtracking_line_search_ex_2_2(
            "gradient", h_ex_2_2, grad_h_ex_2_2, hessian_h_ex_2_2, 1, 0.1, 0.7, 1e-5, np.array([i, j]))
        all_w_newton, _, _ = optimization_with_backtracking_line_search_ex_2_2(
            "Newton", h_ex_2_2, grad_h_ex_2_2, hessian_h_ex_2_2, 1, 0.1, 0.7, 1e-5, np.array([i, j]))
        optimal_w = {
            "gradient": tuple(np.round(all_w_gradient[-1], 2)),
            "Newton": tuple(np.round(all_w_newton[-1], 2))
        }
        for method in ("gradient", "Newton"):
            if optimal_w[method] not in solutions[method]:
                solutions[method][optimal_w[method]] = 0
            solutions[method][optimal_w[method]] += 1

In [ ]:
solutions_np = dict()
for method in ("gradient", "Newton"):
    solutions_keys_np = np.array(list(solutions[method].keys()))
    solutions_values_np = np.array(list(solutions[method].values())).reshape(-1, 1)
    solutions_np[method] = np.hstack((solutions_keys_np, solutions_values_np))

In [ ]:
fig = go.Figure(data=[go.Contour(
    x=w_component_0, y=w_component_1, z=np.log10(h_w),
    hovertemplate="x: %{x:.2f}<br>y: %{y:.2f}<br>z: 10^%{z:.2f} = %{customdata}", customdata=h_w,
    showscale=False
)])
for (method_index, method) in enumerate(("gradient", "Newton")):
    fig.add_scatter(
        x=solutions_np[method][:, 0], y=solutions_np[method][:, 1],
        marker=dict(
            size=5 * np.sqrt(solutions_np[method][:, 2]), color=plotly.colors.qualitative.Set1[method_index]),
        mode="markers",
        hovertemplate="(%{x}, %{y}) found %{customdata} times", name=method + " method",
        customdata=solutions_np[method][:, 2]
    )
fig.update_layout(
    title="Three-hump Camel function - convergence to minima", width=512, height=512, autosize=False)
fig.show()

> Both methods have converged to local minima (rather than the desired global minimum) in around 30\% of the runs.
> The gradient method has never converged to a saddle point; in contrast more than 30\% runs of the Newton method have converged to a saddle point. This observation is more general than the current example, and has been found in several practical applications (especially in machine learning).

## Exercise 2.3
A US university has collected data of past admission into graduate school, and their researchers are interested in how the following variables:
* GRE (Graduate Record Exam) subject tests scores, a standardized test that assesses technical knowledge related to a specific discipline,
* GPA (grade point average) of the candidate, i.e. the average of their marks, and
* prestige of the undergraduate institution that the student attended

are correlated to the admission (or non-admission) of a prospective student. Therefore, we can try model this binary classification problem via a logistic regression.

The response variable is a binary variable (admit/don't admit).

1. Load the dataset of historical data from the [UCLA website](https://stats.idre.ucla.edu/r/dae/logit-regression/).

*Solution*:
> To carry out this task we introduce the [`pandas`](https://pandas.pydata.org/) library, a fast, powerful, flexible and easy to use open source data analysis and manipulation tool.

In [ ]:
import os

In [ ]:
import pandas as pd

> We download the `csv` dataset from the UCLA website.

In [ ]:
if os.path.isfile("data/binary.csv"):
    csv_path = "data/binary.csv"
else:
    # csv_path = "https://stats.idre.ucla.edu/stat/data/binary.csv"
    csv_path = (
        "https://dmf.unicatt.it/~fball/public/optimization_for_machine_learning"
        + "/binary.csv"
    )
df = pd.read_csv(csv_path)

> Once the dataset is imported in pandas, we can print the first few entries of it...

In [ ]:
df

In [ ]:
df.head()

> ... print its main descriptive statistics

In [ ]:
df.describe()

In [ ]:
df.mean()

In [ ]:
df.std()

> ... visualize it by means of histograms

In [ ]:
df.hist()

> ... and much more!

2. GRE and GPA are continuous variables, while the rank of the home university is a categorical variable, where the top-ranked universities are denote by 1 and lowest-ranked institutions by 4. The standard way to handle categorical variables in a regression task is to introduce [dummy variables ](https://en.wikipedia.org/wiki/Dummy_variable_(statistics))
$$r_1, r_2, r_3 \text{ and } r_4$$
where rank 1 universities are represented by
$$r_1 = 1, \quad r_2 = 0, \quad r_3 = 0, \quad r_4 = 0,$$
rank 2 universities are represented by
$$r_1 = 0, \quad r_2 = 1, \quad r_3 = 0, \quad r_4 = 0,$$
and similarly for ranks 3 and 4. In machine learning this process is often called [one-hot encoding](https://en.wikipedia.org/wiki/One-hot).

   One easily notices that introducing all such four variables in a regression task would lead to a linear dependence between the independent variables (features) of the regression. Indeed, one can discard the variable $r_1$ because just by knowning $r_2 = r_3 = r_4 = 0$ we deduce that the student comes from a rank 1 university.
 
   Prepare a new dataset with the following columns: admit, GRE, GPA, rank_2, rank_3 and rank_4.
 
*Solution*:
> `pandas` offer a simple way to generate dummy variables associated to a categorical variable.

In [ ]:
dummy_ranks = pd.get_dummies(df["rank"], prefix="rank")
dummy_ranks.head()

> We then exclude the rank_1 column, and collect all required columns in a new dataset.

In [ ]:
data = df[["admit", "gre", "gpa"]].join(dummy_ranks[["rank_2", "rank_3", "rank_4"]])
data.head()

3. Separate the labels (admit/don't admit) from the features (GRE, GPA, ranks from 2 to 4), as follows.

   Put labels in a vector $\boldsymbol{y}$, where the $j$-th entry stores whether the $j$-th student was admitted or not.

   Put features in a matrix $\boldsymbol{X}$, where the $j$-th row stores the features associated to $j$-th student, ordered as follows: 
   * GRE on the first column, 
   * GPA on the second column, 
   * whether or not they come from a rank 2 university of the third column, 
   * whether or not they come from a rank 3 university of the fourth column, 
   * whether or not they come from a rank 4 university of the fifth column.

*Solution*:
> Note that in the following we will use the capital letter $\boldsymbol{X}$ to the denote the matrix of (all) features, while the lowercase letter $\boldsymbol{x}$ will denote a feature vector (a single student, or a single row of the matrix).

In [ ]:
y = data[["admit"]].to_numpy().reshape(-1)
y

In [ ]:
X = data[["gre", "gpa", "rank_2", "rank_3", "rank_4"]].to_numpy()
X

4. Separate the dataset $(\boldsymbol{X}, \boldsymbol{y})$, composed of 400 rows, in:
   1. a training dataset $(\boldsymbol{X}_{\text{train}}, \boldsymbol{y}_{\text{train}})$, composed of $m_{\text{train}} = 350$ randomly selected rows, and
   2. a test dataset $(\boldsymbol{X}_{\text{test}}, \boldsymbol{y}_{\text{test}})$, composed of the remaining $m_{\text{test}} = 50$ rows.
 
*Solution*:
> In order to randomly select rows, we first define a random permutation of the 400 row indices, using [`numpy.random.permutation`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.permutation.html).

In [ ]:
np.random.seed(23)
perm = np.random.permutation(y.shape[0])
perm

> We then use the first 350 indices in `perm` to define the training set, and the remaining indices to define the test set.

In [ ]:
y_train = y[perm[:350]]
X_train = X[perm[:350]]

In [ ]:
y_test = y[perm[350:]]
X_test = X[perm[350:]]

5. Implement the evaluation of the empirical risk associated to a logistic regression, as well as its gradient and hessian.

*Solution*:
> Similarly to Exercise 1.3, we start by implementing the prediction function $\hat{y}(\boldsymbol{x}; \boldsymbol{w}) = \sigma(\boldsymbol{s}^T \boldsymbol{x} + q)$ associated to a logistic regression with $\boldsymbol{x} \in \mathbb{R}^5$, where $\sigma(z) = \frac{1}{1 + e^{-z}}$ is the sigmoid function, and the weights (optimization variable) are given by
$$\boldsymbol{w} = \begin{bmatrix}\boldsymbol{s}\\ q\end{bmatrix} = \begin{bmatrix}s^{(0)}\\s^{(1)}\\ \dots\\s^{(4)}\\q\end{bmatrix} \in \mathbb{R}^6.$$

In [ ]:
def sigmoid(z: float) -> float:
    """Evaluate the sigmoid function."""
    return 1 / (1 + np.exp(-z))

In [ ]:
def y_hat(x_j: np.ndarray, w: np.ndarray) -> float:
    """Evaluate the prediction function associated to a logistic regression."""
    return sigmoid(np.dot(w[0:5], x_j) + w[5])

> The logistic loss and its derivatives are:
$$\ell(\boldsymbol{x}, y; \boldsymbol{w}) = - y \log \sigma(\boldsymbol{s}^T \boldsymbol{x} + q) - (1 - y) \log (1 - \sigma(\boldsymbol{s}^T \boldsymbol{x} + q)).$$
$$\nabla_\boldsymbol{w} \ell(\boldsymbol{x}, y; \boldsymbol{w}) = (- y + \sigma(\boldsymbol{s}^T \boldsymbol{x} + q)) \begin{bmatrix}\boldsymbol{x}\\1\end{bmatrix}.$$
$$\nabla_\boldsymbol{w}^2 \ell(\boldsymbol{x}, y; \boldsymbol{w}) = \sigma(\boldsymbol{s}^T \boldsymbol{x} + q) (1 - \sigma(\boldsymbol{s}^T \boldsymbol{x} + q)) \begin{bmatrix}\boldsymbol{x} \boldsymbol{x}^T & \boldsymbol{x}\\\boldsymbol{x}^T & 1\end{bmatrix}.$$

In [ ]:
def logistic_loss(x_j: np.ndarray, y_j: float, w: np.ndarray) -> float:
    """Evaluate the logistic loss."""
    return - y_j * np.log(y_hat(x_j, w)) - (1 - y_j) * np.log(1 - y_hat(x_j, w))

In [ ]:
def grad_logistic_loss(x_j: np.ndarray, y_j: float, w: np.ndarray) -> np.ndarray:
    """Evaluate the gradient of the logistic loss."""
    vec = np.zeros((6, ))
    vec[0:5] = x_j
    vec[5] = 1
    return (y_hat(x_j, w) - y_j) * vec

In [ ]:
def hessian_logistic_loss(x_j: np.ndarray, y_j: float, w: np.ndarray) -> np.ndarray:
    """Evaluate the hessian of the logistic loss."""
    mat = np.zeros((6, 6))
    mat[0:5, 0:5] = np.outer(x_j, x_j)
    mat[0:5, 5] = mat[5, 0:5] = x_j
    mat[5, 5] = 1
    return y_hat(x_j, w) * (1 - y_hat(x_j, w)) * mat

> The empirical risk can finally be obtained summing the loss functions corresponding to every row $\boldsymbol{x}_j$ and label $y_j$ of the *training* dataset. In the implementation below, note how the `for` loop over `X_train` will be carried out row by row, which is actually what we need here.

In [ ]:
def f_ex_2_3(w: np.ndarray) -> float:
    """Evaluate the empirical risk."""
    m = X_train.shape[0]
    return 1 / m * sum(logistic_loss(x_j, y_j, w) for (x_j, y_j) in zip(X_train, y_train))

In [ ]:
def grad_f_ex_2_3(w: np.ndarray) -> np.ndarray:
    """Evaluate the gradient of the empirical risk."""
    m = X_train.shape[0]
    return 1 / m * sum(grad_logistic_loss(x_j, y_j, w) for (x_j, y_j) in zip(X_train, y_train))

In [ ]:
def hessian_f_ex_2_3(w: np.ndarray) -> np.ndarray:
    """Evaluate the hessian of the empirical risk."""
    m = X_train.shape[0]
    return 1 / m * sum(hessian_logistic_loss(x_j, y_j, w) for (x_j, y_j) in zip(X_train, y_train))

> We test the implemented functions on a point $\boldsymbol{w}_0 = \boldsymbol{0}$.

In [ ]:
w_0 = np.zeros((6, ))
w_0

In [ ]:
f_ex_2_3(w_0)

In [ ]:
grad_f_ex_2_3(w_0)

In [ ]:
hessian_f_ex_2_3(w_0)

6. Implement a Python function for the minimization of a function $f$ using one of the following methods:
   * gradient descent, or
   * Newton method, or
   * BFGS method,

   with backtracking line search. Such function should:
   * take as input the method to use (`Newton` or `gradient` or `BFGS`), the function $f$, its gradient $\nabla f$, its hessian $\nabla^2 f$, the constants $\alpha$, $c_1$ and $c_2$ of the backtracking algorithm, the tolerance $\varepsilon$ and $K_{\max}$ for the stopping criterion and the initial condition $\boldsymbol{w}_{0}$;
   * return as outputs the optimization variable iterations $\{\boldsymbol{w}_k\}_k$, the corresponding function values $\{f(\boldsymbol{w}_k)\}_k$ and gradients $\{\nabla f(\boldsymbol{w}_k)\}_k$.

   Use the following stopping criteria:
   * the main stopping criterion is based on the norm of the gradient. Continue the iterations while such norm is above $\varepsilon$;
   * allow a maximum number of iterations $K_{\max}$ of iterations, after which the method should terminate with a warning.
 
*Solution*:
> The implementation is very similar to the one in Exercise 2.2, with the additional cases associated to BFGS.

In [ ]:
def optimization_with_backtracking_line_search_ex_2_3(
    method: str, f: typing.Callable, grad_f: typing.Callable, hess_f: typing.Callable, alpha: float,
    c_1: float, c_2: float, epsilon: float, maxit: int, w_0: np.ndarray
) -> typing.Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Run one of gradient descent, Newton or BFGS method with backtracking line search.

    Parameters
    ----------
    method : str
        optimization method to be used, either "gradient", "Newton" or "BFGS".
    f, grad_f, hess_f : Python function
        callable evaluating the cost function, its gradient and its hessian, respectively.
    alpha : float
        initial step length.
    c_1, c_2 : float
        constants of the backtracking algorithm.
    epsilon : float
        tolerance for the stopping criterion on the error on the norm of the gradient of the cost.
    maxit : int
        maximum number of allowed iterations.
    w_0 : 1d numpy array
        numpy array containing the initial condition.

    Returns
    -------
    2d numpy array
        history of the optimization variables iterations.
    1d numpy array
        history of the cost function values.
    2d numpy array
        history of the gradient of the cost function.
    """
    assert method in ("gradient", "Newton", "BFGS")

    # Prepare lists collecting the required outputs over the iterations
    all_w = [w_0]
    all_f = [f(w_0)]
    all_grad_f = [grad_f(w_0)]

    # Prepare iteration counter
    k = 0

    # Print convergence history
    print("Iteration", 0)
    print("\t Cost function", all_f[0])
    print("\t Norm of gradient of cost function", np.linalg.norm(all_grad_f[0]))

    # Prepare initial approximation of inverse of the hessian (BFGS only)
    if method == "BFGS":
        inv_hess_f_k = np.linalg.inv(hess_f(w_0))
        I = np.eye(w_0.shape[0])

    # Use the norm of the gradient as stopping criterion.
    while np.linalg.norm(all_grad_f[k]) > epsilon:
        w_k = all_w[k]
        f_k = all_f[k]
        grad_f_k = all_grad_f[k]

        # Define the search direction p_k
        if method == "gradient":
            p_k = - grad_f_k
            p_k_dot_g_k = - np.linalg.norm(grad_f_k)**2
        elif method == "Newton":
            hess_f_k = hess_f(w_k)
            p_k = - np.linalg.solve(hess_f_k, grad_f_k)
            p_k_dot_g_k = np.dot(p_k, grad_f_k)
        elif method == "BFGS":
            p_k = - np.dot(inv_hess_f_k, grad_f_k)
            p_k_dot_g_k = np.dot(p_k, grad_f_k)

        # Carry out a backtracking line search
        alpha_k = alpha
        while np.isnan(f(w_k + alpha_k * p_k)) or f(w_k + alpha_k * p_k) > f_k + c_1 * alpha_k * p_k_dot_g_k:
            alpha_k = c_2 * alpha_k

        # Compute w_{k+1}
        w_k_plus_1 = w_k + alpha_k * p_k

        # Update required outputs
        all_w.append(w_k_plus_1)
        all_f.append(f(w_k_plus_1))
        all_grad_f.append(grad_f(w_k_plus_1))

        # Update approximation of inverse of the hessian (BFGS only)
        if method == "BFGS":
            y_k = all_grad_f[k + 1] - all_grad_f[k]
            s_k = all_w[k + 1] - all_w[k]
            rho_k = 1 / np.dot(y_k, s_k)
            inv_hess_f_k = (
                np.dot(np.dot(I - rho_k * np.outer(s_k, y_k), inv_hess_f_k), I - rho_k * np.outer(y_k, s_k))
                + rho_k * np.outer(s_k, s_k)
            )

        # Increment iteration counter
        k += 1

        # Print convergence history
        print("Iteration", k)
        print("\t Step length", alpha_k)
        print("\t Cost function", all_f[k])
        print("\t Norm of gradient of cost function", np.linalg.norm(all_grad_f[k]))

        # Bail out if exceeded allowed number of iterations
        if k >= maxit:
            print("WARNING: " + method + " method exceeded number of allowed iterations")
            break

    # For convenience we transform the outputs into numpy array before returning
    return np.array(all_w), np.array(all_f), np.array(all_grad_f)

7. Choose $c_1 = 0.1$, $c_2 = 0.7$, $\varepsilon = 10^{-5}$, $\boldsymbol{w}_0 = \boldsymbol{0}$. Run gradient, Newton and BFGS methods, and compare their convergence rates.

*Solution*:
> Note that the text does not say which value of $\alpha$ to choose. In a typical machine learning scenario, it is up to us to decide the hyperparameters, using our experience on the problem and any insights the theory might offer.
>
> The step length for gradient method suggested by the theory is $\alpha = \frac{1}{L}$. In order to compute the smoothness constant $L$, we have to slightly generalize the formula we have seen in Lecture 0. Indeed, the formula we have seen in the lecture had been derived for the scalar case $x \in \mathbb{R}$, while now we have a vector of features $\boldsymbol{x} \in \mathbb{R}^5$. You may follow the same steps we had there, or use directly the expression of the hessian of the empirical risk to see that
$$L = \frac{1}{4 m_{\text{train}}} \lambda_{\max}(\boldsymbol{X}_{\text{train}}^T \boldsymbol{X}_{\text{train}}).$$


In [ ]:
eigs, _ = np.linalg.eig(np.dot(X_train.T, X_train))
L = np.max(eigs) / (4 * X_train.shape[0])
print("L =", L)
print("1 / L =", 1 / L)

> The suggested value of $\alpha$ is $O(10^{-5})$, which seems like a very small step length. Since we have a backtracking implementation, we may try using a slighlty larger (say, $10^{-4}$) first guess for the step length. If such value is too large the backtracking algorithm will decrease it appropriately.

In [ ]:
all_w_gradient, all_f_gradient, all_grad_f_gradient = optimization_with_backtracking_line_search_ex_2_3(
    "gradient", f_ex_2_3, grad_f_ex_2_3, hessian_f_ex_2_3, 1e-4, 0.1, 0.7, 1e-5, 200, w_0)

> Notice how:
> * the code stopped due to reaching the maximum value of 200 iterations, with gradient norms which are considerably above the required tolerance of $\varepsilon = 10^{-5}$;
> * our larger step length $\alpha = 10^{-4}$ was always rejected. Actual values of chosen step lengths are between $1 \cdot 10^{-5}$ and $4 \cdot 10^{-5}$, which are of the same order of magnitude as the suggested value of $1/L$;
> * after a reasonable large decrease of the cost function between iterations 0 and 4, the improvements from iterations 4 to 200 are negligible. The gradient method is showing a very slow sublinear convergence. Indeed, since logistic regression is convex (but not strongly convex), in general we cannot expect a linear convergence. From Lecture 1 we know that the number of iterations required to reach the tolerance $\varepsilon$ on the gradient norm scales as
$$ O\left(\frac{L}{\varepsilon^2}\right).$$
Since $L \approx 10^{5}$ and $\varepsilon = 10^{-5}$, the theory warns us that it might take
$$ O(10^{15}) $$
iterations to convergence to the desired tolerance. No practical application can wait for such a large number of iterations.
>
> Therefore, we try to use alternative methods, such as Newton and BFGS. We use $\alpha = 1$ for both, as convergence results suggest to start backtracking from a unit step.

In [ ]:
all_w_newton, all_f_newton, all_grad_f_newton = optimization_with_backtracking_line_search_ex_2_3(
    "Newton", f_ex_2_3, grad_f_ex_2_3, hessian_f_ex_2_3, 1, 0.1, 0.7, 1e-5, 200, w_0)

In [ ]:
all_w_bfgs, all_f_bfgs, all_grad_f_bfgs = optimization_with_backtracking_line_search_ex_2_3(
    "BFGS", f_ex_2_3, grad_f_ex_2_3, hessian_f_ex_2_3, 1, 0.1, 0.7, 1e-5, 200, w_0)

> Both methods have converged very fast, in less than 10 iterations. BFGS uses only a handful of iterations more than Newton, but does not require the evaluation of the hessian matrix (for simplicity we used it to initialize $\boldsymbol{H}_0$, but in actual implementations this initialization would be done differently) and does not require to solve any linear system, so it might be preferable when dealing with a number of features which is very large (much more than 5 features we have here).
>
> This can be confirmed by the following plot. Uncomment the line which restricts the range of the horizontal axis for a clearer comparison between Newton and BFGS.

In [ ]:
fig = go.Figure()
all_methods = ["Gradient", "Newton", "BFGS"]
all_norm_grad_f_method = [
    np.linalg.norm(all_grad_f_gradient, axis=1),
    np.linalg.norm(all_grad_f_newton, axis=1),
    np.linalg.norm(all_grad_f_bfgs, axis=1)
]
for method_index in range(3):
    fig.add_scatter(
        x=np.arange(all_norm_grad_f_method[method_index].shape[0]), y=all_norm_grad_f_method[method_index],
        marker=dict(color=plotly.colors.qualitative.Set1[method_index], size=10),
        line=dict(color=plotly.colors.qualitative.Set1[method_index], width=2),
        mode="lines+markers", name=all_methods[method_index] + " method"
    )
fig.update_layout(
    title="Logistic regression - error on the function value - different methods",
    width=768, height=768, autosize=False
)
# fig.update_xaxes(range=[0, 10])
fig.update_yaxes(type="log", exponentformat="power")
fig.show()

> The final weights provided by Newton and BFGS are comparable. (Note: the values obtained here are slightly different from the ones reported on the UCLA website because the ones on the website have been computed using the whole dataset $(\boldsymbol{X}, \boldsymbol{y})$, and not just a subset $(\boldsymbol{X}_{\text{train}}, \boldsymbol{y}_{\text{train}})$).

In [ ]:
print(all_w_newton[-1])
print(all_w_bfgs[-1])

> We may try to interpret these weights to draw some conclusions on how the features affect the prediction:
> * the third, fourth and fifth weights are multiplied by the $r_2$, $r_3$ and $r_4$ dummy variables, and they have a quite clear statistical interpretation. Since $r_2 = r_3 = r_4 = 0$ means that a student comes from a rank 1 university, we will interpret these coefficients using the rank 1 institution as baseline. The third, fourth and fifth weights are negative, meaning that not being from a rank 1 university negatively affects the odds of a student to be admitted (reasonable!). Furthermore, since the coefficients are decreasing when moving from the third to the fifth, the lower the rank of the home institution, the worse are the odds of being admitted. The weight associated to $r_3$ is roughly twice the one associated to $r_2$, while the coefficient associated to $r_4$ is roughly 2.3 times the one to $r_2$;
> * the first and second weights are associated to the GRE and GPA respectively. The first weight is O(100) times smaller than the second one. Does this mean that the GPA has a large influence on the odds of getting admitted, while the GRE has negigible influence? Are you sure of this conclusion?

8. Using the optimal weights, assess the accuracy of the prediction on the test dataset.

*Solution*:
> Since this is a classification task (expect labels are admit/don't admit), given a feature vector $\boldsymbol{x}$ in the test set $\boldsymbol{X}_{\text{test}}$, we assign the label ''admit'' if
> $\hat{y}(\boldsymbol{x}; \boldsymbol{w}) = \sigma(\boldsymbol{s}^T \boldsymbol{x} + q) > 0.5$
> where $\boldsymbol{w}$ are the optimal weights computed (e.g.) by the Newton method.
>
> We then compare the predicted values and the actual values (stored in $\boldsymbol{y}_{\text{test}}$) by means of the following *table of confusion*
> <table>
    <tr>
        <th></th>
        <th>$\hat{y} \leq 0.5$</th>
        <th>$\hat{y} > 0.5$</th>
    </tr>
    <tr>
        <td>$y = 0$</td>
        <td><i># of true negative</i></td>
        <td># of false positive</td>
    </tr>
    <tr>
        <td>$y = 1$</td>
        <td># of false negative</td>
        <td><i># of true positive</i></td>
    </tr>
> </table>
> and the corresponding accuracy index
$$\text{accuracy} = \frac{\text{# of true positive} + \text{# of true negative}}{\text{# of true positive} + \text{# of false negative} + \text{# of false positive} + \text{# of true negative}}$$



In [ ]:
confusion = np.zeros((2, 2))
for (x_j, y_j) in zip(X_test, y_test):
    y_hat_j = y_hat(x_j, all_w_newton[-1])
    confusion[int(y_j > 0.5), int(y_hat_j > 0.5)] += 1
confusion

In [ ]:
accuracy = (confusion[0, 0] + confusion[1, 1]) / (
    confusion[0, 0] + confusion[0, 1] + confusion[1, 0] + confusion[1, 1])
accuracy

> The trained model handles very well the true negative cases (i.e., students who were not admitted would have indeed been predicted as not admitted), but also has several false negative (i.e., students who were admitted, but the model would have predicted as not admitted). The resulting accuracy on the test set is $80\%$.

9. GRE scores are between 220 and 990, while GPA values are between 0 and 4. Repeat the optimization after having normalized the GRE and GPA columns to be between 0 and 1. How does this normalization process affect the convergence of the optimization method and the interpretation of the optimal weights?

*Solution*:
> We normalize the first two columns of $\boldsymbol{X}_{\text{train}}$ and $\boldsymbol{X}_{\text{test}}$. Note that we are modifying the matrices `X_train` and `X_test` *in place*, not creating a new copy. Be careful: do not execute the cell below twice (you would be applying the scaling process twice!), and do not re-run cells at point 5 (if you need to do so, make sure to reload the original matrix `X_train` and `X_test` re-running the corresponding cell at point 2).

In [ ]:
X_train[:, 0] -= 220
X_train[:, 0] /= (990 - 220)
X_train[:, 1] /= 4

In [ ]:
X_train

In [ ]:
X_test[:, 0] -= 220
X_test[:, 0] /= (990 - 220)
X_test[:, 1] /= 4

> Since the matrix $\boldsymbol{X}_{\text{train}}$ has changed, we need to recompute the smoothness constant $L$ of the empirical risk function. Note how after normalization the smoothness constant is $O(0.1)$, which is considerably lower than the $O(10^{5})$ we had computed without normalization.

In [ ]:
eigs, _ = np.linalg.eig(np.dot(X_train.T, X_train))
L = np.max(eigs) / (4 * X_train.shape[0])
print("L =", L)
print("1 / L =", 1 / L)

> Having a different value of $L$ also means that the suggested value of step length $\alpha$ is now bigger when applying the gradient method. We try backtracking from an initial step length of 10.

In [ ]:
all_w_gradient, all_f_gradient, all_grad_f_gradient = optimization_with_backtracking_line_search_ex_2_3(
    "gradient", f_ex_2_3, grad_f_ex_2_3, hessian_f_ex_2_3, 10, 0.1, 0.7, 1e-5, 200, w_0)

> Execution of the gradient method stopped having exceeded the maximum number of iterations. However, the results at the final iteration are considerably better than the ones we have obtained at step 5 (non normalized dataset). Compare for instance the cost and the norm of the gradient at the final iteration.
> This is a concrete machine learning problem that (as we already discussed in Exercise 2.1) shows how first order methods are sensitive to different scales (orders of magnitude) in the optimization variables. This is (part of) the reason why many machine learning algorithms suggest to always normalize/standardize the data!
>
> Are second order methods affected by variable scaling too?

In [ ]:
all_w_newton, all_f_newton, all_grad_f_newton = optimization_with_backtracking_line_search_ex_2_3(
    "Newton", f_ex_2_3, grad_f_ex_2_3, hessian_f_ex_2_3, 1, 0.1, 0.7, 1e-5, 200, w_0)

In [ ]:
all_w_bfgs, all_f_bfgs, all_grad_f_bfgs = optimization_with_backtracking_line_search_ex_2_3(
    "BFGS", f_ex_2_3, grad_f_ex_2_3, hessian_f_ex_2_3, 1, 0.1, 0.7, 1e-5, 200, w_0)

> Second order methods are either not affected by scaling at all (Newton method) or only very mildly affected by it (BFGS). This is surely another advantage of second order methods compared to first order ones.

In [ ]:
fig = go.Figure()
all_methods = ["Gradient", "Newton", "BFGS"]
all_norm_grad_f_method = [
    np.linalg.norm(all_grad_f_gradient, axis=1),
    np.linalg.norm(all_grad_f_newton, axis=1),
    np.linalg.norm(all_grad_f_bfgs, axis=1)
]
for method_index in range(3):
    fig.add_scatter(
        x=np.arange(all_norm_grad_f_method[method_index].shape[0]), y=all_norm_grad_f_method[method_index],
        marker=dict(color=plotly.colors.qualitative.Set1[method_index], size=10),
        line=dict(color=plotly.colors.qualitative.Set1[method_index], width=2),
        mode="lines+markers", name=all_methods[method_index] + " method"
    )
fig.update_layout(
    title="Logistic regression - error on the function value - different methods",
    width=768, height=768, autosize=False
)
# fig.update_xaxes(range=[0, 10])
fig.update_yaxes(type="log", exponentformat="power")
fig.show()

> We finally print the optimal weights.

In [ ]:
print(all_w_newton[-1])
print(all_w_bfgs[-1])

> Notice how only the first two weights have changed, as we had only scaled the first two variables. Without normalization, it is not fair to compare different weights and conclude whether "one feature is more important than the other", because an increase of 1 GRE point (out of 990) is not comparable to an increase of 1 GPA point (out of 4). It is only possible to carry out a fair comparison on normalized features. Based on the first two values of the weights vector, we conclude that the normalized GPA has an influence which is roughly the double of the one of the normalized GRE on the odds of admission to the graduate program.
>
> Normalization (standardization) has had two beneficial effects:
> * from the optimization point of view, it has improved the convergence of the first order method;
> * from the statistics/machine learning point of view, it has improved the interpretability of the resulting model.

10. Using the optimal weights, assess the accuracy of the prediction on the normalized test dataset.

*Solution*:

In [ ]:
confusion = np.zeros((2, 2))
for (x_j, y_j) in zip(X_test, y_test):
    y_hat_j = y_hat(x_j, all_w_newton[-1])
    confusion[int(y_j > 0.5), int(y_hat_j > 0.5)] += 1
confusion

In [ ]:
accuracy = (confusion[0, 0] + confusion[1, 1]) / (
    confusion[0, 0] + confusion[0, 1] + confusion[1, 0] + confusion[1, 1])
accuracy

> In this case normalization did not make any difference in terms of accuracy. However, in practical cases normalization often helps in obtaining faster convergence of the weights to the optimum (as it happened for us for the gradient method) and, especially when the number of iterations is limited due to comptational resources, normalization often helps in obtaining better models.